In [2]:
#from methods import *
import re
import benepar
import pandas as pd
import nltk
import nltk.draw
from nltk.tree import Tree
from nltk.draw.tree import TreeView
from nltk.stem import WordNetLemmatizer as wln
import os
from IPython.display import display
# import spacy
from  icecream import ic
from scipy.stats import chisquare
from nltk.corpus import wordnet as wn
from flair.data import Sentence
from flair.nn import Classifier
import numpy as np
from nltk.data import find
from nltk.test.gensim_fixt import setup_module
from nltk.corpus import stopwords
import gensim
import gensim.downloader
import pytest
nltk.download('wordnet')
nltk.download('stopwords')


/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/oliverweizel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oliverweizel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
sentence_enders = {".", '?', '!', ':', ';'}
sentential_tags = {'S', 'SBAR', 'SINV', 'SBARQ', 'SQ'}
negators = {'not', 'n\'t'}
verb_tags = {'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'VB', 'MD', 'VP'}
adverb_tags = {'RB', 'RBR', 'RBS', 'WRB', 'ADVP'}
adj_tags = {'JJ', 'JJR', 'JJS', 'ADJP'}
noun_tags = {'NN', 'NNP', 'NNS', 'NNPS', 'NP'}
tensed = {'have', 'do', 'will', 'would', 'can', 'could', 'may', 'might', 'should', 'ought', 'must', 'shall', 'be', 'dare'}
wh_tags = {'WDT', 'WP', 'WP$', 'WRB'}
mod_tags = {'JJ', 'JJR', 'JJS', 'DT', 'PDT', 'CD'}
prep_tags = {'IN', 'TO', 'PP'}

setup_module()
tagger = Classifier.load('sentiment')
#word2vec_sample = str(find('word2vec-google-news-300'))
#model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)
model = gensim.downloader.load('word2vec-google-news-300')

stop_words = set(stopwords.words('english'))

In [4]:
vad_df = pd.read_csv('NRC_EmoLex_NRC-VAD-Lexicon.tsv', sep='\t')
vad_df.columns = ['word', 'valence', 'arousal', 'dominance']
vad_df = vad_df.set_index('word')
vad_df = vad_df.T.to_dict('list')
vad_dict = {k : np.array(vad_df[k]) for k in vad_df}
#print(vad_df.head())
print(vad_dict)

{'aaaah': array([0.52 , 0.636, 0.282]), 'aardvark': array([0.427, 0.49 , 0.437]), 'aback': array([0.385, 0.407, 0.288]), 'abacus': array([0.51 , 0.276, 0.485]), 'abalone': array([0.5  , 0.48 , 0.412]), 'abandon': array([0.052, 0.519, 0.245]), 'abandoned': array([0.046, 0.481, 0.13 ]), 'abandonment': array([0.128, 0.43 , 0.202]), 'abashed': array([0.177, 0.644, 0.307]), 'abate': array([0.255, 0.696, 0.604]), 'abatement': array([0.388, 0.338, 0.336]), 'abba': array([0.562, 0.5  , 0.48 ]), 'abbey': array([0.58 , 0.367, 0.444]), 'abbot': array([0.427, 0.321, 0.483]), 'abbreviate': array([0.531, 0.375, 0.33 ]), 'abbreviation': array([0.469, 0.306, 0.345]), 'abdomen': array([0.469, 0.462, 0.471]), 'abdominal': array([0.49 , 0.456, 0.445]), 'abduct': array([0.173, 0.72 , 0.615]), 'abduction': array([0.062, 0.99 , 0.673]), 'aberrant': array([0.146, 0.765, 0.431]), 'aberration': array([0.125, 0.816, 0.417]), 'abeyance': array([0.33 , 0.51 , 0.292]), 'abhor': array([0.125, 0.602, 0.349]), 'abhor

In [5]:
# takes in an array of english words and returns true if one of a 
# set of words is present
def english_indicator(text, options=['only', 'just']):
  for w in text:
    if w in options:
      return False
  return True

# takes in a line of english text, splits the line into paragraphs
# returns a line splinto sentences as an array of array of strings
# if containing all alphanumeric characters, or if a specified
# other condition is met
def english_processor(text, specific_processor=None):
  res = []

  #https://stackoverflow.com/questions/25735644/python-regex-for-splitting-text-into-sentences-sentence-tokenizing
  sentences = re.split(r'(?<!\w\.\w.)(?<!\b[A-Z][a-z]\.)(?<![A-Z]\.)(?<=\.|\?)\s|\\n', text)

  sentences = [s.split(' ') for s in sentences]


  sentences = [[w.lower() for w in s if w not in { '\n', '<p>', '', ',', '.', '\"', '\'', '?', '!', ':', ';'}] for s in sentences]

  sentences = [s for s in sentences if '@' not in s]

  sentences = [s for s in sentences]

  for s in sentences:
    if not s:
      continue
    if '@' in s:
      continue
    joined = ''.join(s)
    if '#' in joined:
      continue
    if ':' in joined:
      continue
    if ';' in joined:
      continue
    if '-' in joined:
      continue
    res.append(s)
  return res

def new_processor(text, specific_processor=None):
  res = nltk.sent_tokenize(text)
  ic(res)
  return res


# takens in the path to a .txt file, a function that takes in a 
# line containing multiple sentences and returns array of array
# of words in sentences, and a function that return true iff a 
# sentence is valid
def valid_sentences(textpath, processor, indicator):
  res = []
  i = 0
  with open(textpath, 'r') as f:
    for line in f:
      #print(line)
      sentences = [s for s in processor(line) if indicator(s)]
      res = res + sentences
  return res

def pp(t):
  return str(t.flatten()).replace('\n', ' ').replace('   ', ' ')

def text(t):
  return ' '.join(t.leaves())

def conjoin(a, b, left):
  ic("CONJOIN")
  res = None
  if left:
    res = '(' + a.label() + ' (' + str(a) + ' ' + str(b) + ') )'
  else:
    res = '(' + b.label() + ' (' + str(a) + ' ' + str(b) + ')'
  print(res)
  return Tree(res)
  
def lowest_pos(t):
  return [s for s in t.subtrees(lambda t: t.height() == 2)]

def lowest_leaves(t):
  return [t[s] for s in t.subtrees(lambda t: t.height() == 2)]
#returns if corrective, anchored (verb negated)
def find_s(t, pos0):
  # sentential_tags = {'S', 'SBAR', 'SINV', 'SBARQ', 'SQ'}
  # negators = {'not', 'n\'t'}
  # verb_tags = {'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}
  # tensed = {'have', 'do', 'will', 'would', 'can', 'could', 'may', 'might', 'should', 'ought', 'must', 'shall', 'be'}
  #print('t: ' + str(t))
  
  if type(t[pos0]) is str:
    return (False, False)
  print("TP?: " + str(t[pos0].leaf_treeposition(0)))
  print('pos0: ' + text(t[pos0]))
  pos = pos0
  try:
    while t[pos].label() not in sentential_tags:
      pos = pos[:-1]
      if pos == ():
        return (False, False)
  except:
    return (False, False)
  #print('afterpos: ' + str(pos))
  poses = lowest_pos(t[pos])
  i = None
  #print('poses: '+ str(poses))
  #print(poses)
  try:
    for k in range(poses.index(t[pos0])):
      if text(t[poses[k]]) in negators:
        i = k
        break
  except:
    return (False, False)
  if i == None:
    return (False, False)
  try:
    temp = wln().lemmatize(text(t[poses[i-1]]),pos='v')
    if t[poses[i-1]].label() in verb_tags and temp in tensed and temp != 'VB':
      return (True, True)
    return (True, False)
  except:
    return (False, False)

def negation(t, pos0):
  if type(t[pos0]) is str:
    return (False, False)
  #print('pos0: ' + text(t[pos0]))
  pos = pos0
  location = pos0
  try:
    while t[pos].label() not in sentential_tags:
      pos = pos[:-1]
      location = location[1:]
      if pos == ():
        return (False, False)
  except:
    return (False, False)
  poses = t[pos].treepositions('leaves')
  leafs = t[pos].leaves()
  for i, p in enumerate(poses):
    #print("YOOO")
    #print(leafs[i])
    if p == location:
      print("LOCATED: " + str(location))
      return (False, False)
    if leafs[i] in negators:
      ic("WE FOUND ONE")
      try:
        temp = wln().lemmatize(text(t[poses[i-1]]),pos='v')
        ic(temp)
        if t[poses[i-1]].label() in verb_tags and temp in tensed and temp != 'VB':
          return (True, True)
        return (True, False)
      except:
        return (False, False)
  return (False, False)
      
def hope(t, pos0):
  if type(t[pos0]) is str:
    #print("string error")
    return (False, False)
  poses = [p[:-1] for p in t.treepositions('leaves')]
  # print(poses)
  # print(pos0)
  try:
    i = poses.index(pos0)
  except:
    #print("not leaf")
    return (False, False)
  #print("leaf found")
  poses = poses[:i]
  leafs = t.leaves()[:i]
  # print(len(poses))
  # print(len(leafs))
  # print(poses)
  # print(leafs)
  s_pos = lowest_s(t, pos0)
  if s_pos is None:
    return (False, False)
  s_length = len(s_pos)
  # print(text(t[pos0]))
  # print(i)
  seen_neg = False
  for k in range(i):
    # print("lk: " + leafs[k])
    if poses[k][:s_length] == s_pos:
      if k == 0 and leafs[k] in negators:
        #print("initial negator")
        return (True, False)
      if leafs[k].lower() in negators:
        seen_neg = True
      temp_text = wln().lemmatize(text(t[poses[k-1]]),pos='v')
      temp_tag = t[poses[k-1]].label()
      # print("temp_text: " + str(temp_text) + " ; text: " + text(t[poses[k]]))
      # print(temp_tag in verb_tags)
      # print(temp_text in tensed)
      # print(temp_tag != 'VB')
      # print(leafs[k])
      # print(leafs[k] in negators)
      if temp_tag in verb_tags and temp_text in tensed and temp_tag != 'VB' and leafs[k].lower() in negators:
        return (True, True)
  return (seen_neg, False)


def lowest_s(t, pos0):
  pos = pos0
  #print(t[pos])
  #print('starter POS: ' + str(pos))
  try:
    while t[pos].label() not in sentential_tags:
      #print('POS loop: ' + str(pos))
      pos = pos[:-1]
  except:
    return None
  
  return pos

def id(t, d):
  return -1

def funnel(tag):
  if tag in verb_tags:
    return "V"
  elif tag in adverb_tags:
    return 'R'
  elif tag in adj_tags:
    return 'J'
  elif tag in noun_tags:
    return 'N'
  elif tag in sentential_tags:
    return 'S'
  elif tag in prep_tags:
    return 'P'
  else:
    return 'X'


def sent_apply(r):
  s0 = Sentence(r)
  tagger.predict(s0)
  sentiment = s0.labels[0].value
  score = s0.labels[0].score
  if sentiment == 'NEGATIVE':
    return -1 * score
  return score

def cosine_apply(p, q):
  try:
    res = model.similarity(p, q)
    return res
  except:
    return np.NaN
  
def avg_vad(t):
  #s = Tree(t).leaves()
  s = t.split()
  total = np.array([0.0,0.0,0.0])
  n = 0
  for w in s:
    if w not in stop_words and w in vad_dict.keys():
      n += 1
      total += vad_dict[w]
  if n == 0:
    return np.array([np.nan, np.nan, np.nan])
  return total / n

def apply_vad(r):
  rt = r.tolist()
  return r[0], r[1], r[2]


In [6]:
parser = benepar.Parser("benepar_en3")
ans = valid_sentences('COCA/text_magazine_qep/w_mag_2006.txt', english_processor, english_indicator)[:100]
df = pd.DataFrame({"sentence":ans})
tree_gen = parser.parse_sents([benepar.InputSentence(s) for s in ans])
trees = []
for t in tree_gen:
  trees.append(t)

/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
/U

In [7]:
txt1 = "I ate not an apple but a pear"
bs = parser.parse_sents(txt1)
ok = []
for t in bs:
  #print(t)
  for tp in t.treepositions():
    #print("YU")
    #print(tp)
    #print(t[tp])
    print(hope(t, tp))

(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(False, False)
(True, False)
(False, False)
(True, False)
(False, False)
(True, False)
(False, False)
(False, False)
(True, False)
(False, False)
(True, False)
(False, False)


In [8]:
parent_tree = []
parent_text = []
parent_type = []
left_tree = []
left_text = []
left_type = []
cc_tree = []
cc_text = []
right_tree = []
right_text = []
right_type = []
corrective = []
anchored = []
clausal_coord = []
base_text = []

count = -1
for t in trees:

  count += 1
  # if count % 10 == 0:
  print(count)
  for tp in t.treepositions():
    if isinstance(t[tp], str):
      continue
    if t[tp].label() == 'CC':
      parent = t[tp[:-1]]
      relative_tp = tp[-1]
      if relative_tp + 1 >= len(parent) or relative_tp - 1 <= -1:
        continue
      left = parent[relative_tp - 1]
      conj = parent[relative_tp]
      right = parent[relative_tp + 1]
      cc = False
      if left.label() in sentential_tags and right.label() in sentential_tags:
        cc = True
      
      if 'only' in left.leaves() or 'just' in left.leaves():
        continue
      if text(left) == 'but':
        continue
      if left.label() == 'CC':
        continue

      corr, anch = hope(t, tp)
      left_i = 1
      new_left = ""
      most_recent = None
      quit = False
      while True:
        if parent[relative_tp - left_i].label() not in adverb_tags:
          quit = True
        new_left = str(parent[relative_tp - left_i]) + new_left
        most_recent = parent[relative_tp - left_i]
        left_i += 1
        if relative_tp - left_i < 0:
          break
        if quit:
          break
      if left_i > 1:
        left = Tree.fromstring('(' + most_recent.label() + new_left + ')')

      right_i = 1
      new_right = ""
      most_recent = None
      quit = False
      while True:
        if parent[relative_tp + right_i].label() not in adverb_tags:
          quit = True
        new_right =  new_right + str(parent[relative_tp + right_i])
        most_recent = parent[relative_tp + right_i]
        right_i += 1
        if relative_tp + right_i >= len(parent):
          break
        if quit:
          break
      if right_i > 1:
        right = Tree.fromstring('(' + most_recent.label() + new_right + ')')




      # try:
      #   farright = parent[relative_tp + 2]
      #   if right.label() == 'ADVP' and farright.label() != 'ADVP':
      #     # ic(text(parent))
      #     # ic("HI")
      #     right = conjoin(right, farright, False)
      #     # ic("YO")
      #     # ic(text(right))
      # except:
      #   pass
      

      parent_tree.append(parent)
      parent_text.append(text(parent))
      parent_type.append(parent.label())
      left_tree.append(left)
      left_text.append(text(left))
      left_type.append(left.label())
      right_tree.append(right)
      right_text.append(text(right))
      right_type.append(right.label())
      cc_tree.append(conj)
      cc_text.append(text(conj))
      corrective.append(corr)
      anchored.append(anch)
      clausal_coord.append(cc)
      base_text.append(text(t))


24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [9]:
df = pd.DataFrame({'parent_tree':parent_tree, 'parent_text':parent_text, 'parent_type':parent_type, 'left_tree':left_tree, 'left_text':left_text, 'left_type':left_type, 'right_tree':right_tree, 'right_text':right_text, 'right_type':right_type, 'cc_tree':cc_tree, 'cc_text':cc_text})
df['base_text'] = base_text
df['parent_funnel'] = df.apply(lambda row: funnel(row['parent_type']), axis=1)
df['left_funnel'] = df.apply(lambda row: funnel(row['left_type']), axis=1)
df['right_funnel'] = df.apply(lambda row: funnel(row['right_type']), axis=1)
df['same_type'] = df.apply(lambda row: row['left_type'] == row['right_type'], axis=1)
df['same_funnel'] = df.apply(lambda row: row['left_funnel'] == row['right_funnel'], axis=1)
df['clausal_coord'] = clausal_coord
df['corrective'] = corrective
df['anchored'] = anchored
df['left_sent'] = df.apply(lambda row: sent_apply(row['left_text']), axis=1)
df['base_sent'] = df.apply(lambda row: sent_apply(row['base_text']), axis=1)
df['right_sent'] = df.apply(lambda row: sent_apply(row['right_text']), axis=1)
df['cosine'] = df.apply(lambda row: cosine_apply(row['left_text'], row['right_text']), axis=1)
df['left_vad'] = df.apply(lambda row: avg_vad(row['left_text']), axis=1)
df['left_valence'] = df.apply(lambda row : row['left_vad'][0], axis=1)
df['left_arousal'] = df.apply(lambda row : row['left_vad'][1], axis=1)
df['left_dominance'] = df.apply(lambda row : row['left_vad'][2], axis=1)
df['right_vad'] = df.apply(lambda row: avg_vad(row['right_text']), axis=1)
df['right_valence'] = df.apply(lambda row : row['right_vad'][0], axis=1)
df['right_arousal'] = df.apply(lambda row : row['right_vad'][1], axis=1)
df['right_dominance'] = df.apply(lambda row : row['right_vad'][2], axis=1)
df['vad_cosine'] = df.apply(lambda row: np.dot(row['left_vad'], row['right_vad'])/(np.linalg.norm(row['left_vad'])*np.linalg.norm(row['right_vad'])), axis=1)
df['vad_distance'] = df.apply(lambda row: np.linalg.norm(row['left_vad'] - row['right_vad']), axis=1)
print(df['cc_text'].value_counts())
res = chisquare(df['cc_text'].value_counts())
print(res)
print(df.head())

cc_text
and    29
but     5
yet     1
Name: count, dtype: int64
Power_divergenceResult(statistic=39.314285714285724, pvalue=2.9040983974372417e-09)
                                         parent_tree  \
0  [[[(PRP he)], [(VBD did), (RB n't), (VP (VB ma...   
1  [[[(NP (PRP he)), (VP (VBD had) (VP (VBN misse...   
2                           [[left], [and], [right]]   
3  [[[(NN muthiya)], [(MD would), (VP\n  (VB miss...   
4                      [[class], [and], [composure]]   

                                         parent_text parent_type  \
0  he did n't make it to the weekend but simply b...           S   
1     he had missed but right in the thick of things           S   
2                                     left and right        ADJP   
3  muthiya would miss the cut by a dozen strokes ...           S   
4                                class and composure          NP   

                                           left_tree  \
0  [[[(PRP he)], [(VBD did), (RB n't), (VP (VB ma.

In [10]:
df.to_csv('out8.csv', index=False)